In [1]:
import os
import requests
import json
import pandas as pd
import numpy as np
import PyPDF2
from tqdm.notebook import tqdm
from sklearn.metrics.pairwise import cosine_similarity

from openai import OpenAI
from sentence_transformers import SentenceTransformer, losses, InputExample
from sentence_transformers.evaluation import InformationRetrievalEvaluator
import torch
from torch.utils.data import DataLoader

In [2]:
model = SentenceTransformer('BAAI/bge-m3')

In [3]:
train_examples = [
    InputExample(texts=['AI란 무엇인가?', 'AI는 인간의 지능을 모방한 기술입니다.', 'AI는 로봇과 같은 물리적 형태를 가진 기계입니다.']),
    InputExample(texts=['딥러닝이란?', '신경망을 여러 층 쌓아 데이터로부터 학습하는 기계학습 방법입니다.', '컴퓨터가 스스로 생각하는 방법입니다.']),
    InputExample(texts=['Python은 어디에 쓰이나요?', 'Python은 데이터 분석, 웹 개발, AI 등에 널리 사용됩니다.']),
    InputExample(texts=['자연어 처리란?', '컴퓨터가 인간의 언어를 이해하고 처리하는 AI의 한 분야입니다.']),
]

In [4]:
batch_size = 32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

In [5]:
loss = losses.MultipleNegativesRankingLoss(model, scale=20)

train_loss = losses.MultipleNegativesRankingLoss(model)
warmup_steps = int(len(train_dataloader) * 0.1)

In [6]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs = 3,
    warmup_steps=warmup_steps,
    optimizer_params={'lr':1e-5},
    output_path='../../../exercisebook_large_data/korean_sentence-embedding-model'
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


In [7]:
def extract_text_from_pdf(pdf_path):
    text_chunks = []
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            if text.strip():
                text = text.strip()
                if len(text) > 10:
                    text_chunks.append(text)
    
    return text_chunks

In [8]:
train_corpus = extract_text_from_pdf('data/ict_usa_2024.pdf')
print(f'학습 데이터 문서 개수: {len(train_corpus)}')

val_corpus = extract_text_from_pdf('data/ict_japan_2024.pdf')
print(f'검증 데이터 문서 개수: {len(val_corpus)}')

학습 데이터 문서 개수: 26
검증 데이터 문서 개수: 27
